In [102]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import datetime,time,warnings,itertools,os,sys,math,platform

import pandas as pd

import plotly.express as px

In [11]:
if platform.system()=='Darwin':
    filepath = '../../../../../../TF_data/HEALTHCARE/'
else:
    filepath = 'TBD'

In [64]:
# Base files

# Access
sf_a = pd.read_csv(filepath+'healthcare-access.csv')

# Costs
sf_c = pd.read_csv(filepath+'healthcare-costs.csv',sep=';').drop(columns=['Unnamed: 0','Unnamed: 6'])

# Quality
sf_q = pd.read_csv(filepath+'healthcare-quality.csv',sep=';').rename(columns={'Unnamed: 0':'Country'}).drop(columns=['Unnamed: 2','Unnamed: 3'])

In [114]:
# Preprocessing

df_a = sf_a[sf_a['Year']==2015].drop(columns=['Code','Year']).rename(columns={'Entity':'Country','HAQ Index (IHME (2017))':'Access'})
df_a['Country'] = df_a['Country'].str.strip()

df_c = sf_c[['Country','2018']]
df_c['Country'] = df_c['Country'].apply(lambda x: str(x)[:-1])
df_c['2018'] = df_c['2018'].apply(lambda x: np.double(x.replace(',','')))
df_c = df_c.rename(columns={'2018':'Costs'})
df_c['Country'] = df_c['Country'].str.strip()

df_q = sf_q.copy().rename(columns={'2019 Health Care Index':'Quality'})
df_q['Country'] = df_q['Country'].str.strip()

/var/folders/3l/h5txsrx128z_kjh049xwhtqh0000gn/T/ipykernel_1414/1799433717.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3l/h5txsrx128z_kjh049xwhtqh0000gn/T/ipykernel_1414/1799433717.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
# Synthesize
df = df_a.merge(df_c,left_on='Country',right_on='Country',how='outer')
df = df.merge(df_q,left_on='Country',right_on='Country',how='outer')

In [116]:
df = df[(~df['Access'].isna())&(~df['Costs'].isna())&(~df['Quality'].isna())]

In [117]:
# Normalize

df['Access'] = df['Access'].apply(lambda x: x/100)

c_min = df['Costs'].min()
c_max = (df['Costs']-c_min).max()
df['Costs'] = df['Costs'].apply(lambda x: (x-c_min)/c_max)

df['Quality'] = df['Quality'].apply(lambda x: x/100)

In [120]:
px.scatter_ternary(
    df,
    a='Access',
    b='Costs',
    c='Quality',
    hover_name='Country'
)